In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
dados =pd.read_csv('databasePJF.csv', header=0, sep=',', quotechar='"')

In [4]:
dados

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.00,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.00,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.00,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.00,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.00,40348.79,0,0


In [52]:
#criando um dataframe com menos não fraudes para tornar maior a proporção de fraudes para não fraudes.
dadosfraud = dados[(dados.isFraud == 1)]
dadosnfraud = dados[(dados.isFraud == 0)]
dadosnfraud = dadosnfraud.sample(frac = 1)
dadosnfraud = dadosnfraud.iloc[0:25000]
dadosfinais = pd.concat([dadosfraud,dadosnfraud])
dadosfinais

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
251,1,TRANSFER,2806.00,C1420196421,2806.00,0.00,C972765878,0.00,0.00,1,0
252,1,CASH_OUT,2806.00,C2101527076,2806.00,0.00,C1007251739,26202.00,0.00,1,0
680,1,TRANSFER,20128.00,C137533655,20128.00,0.00,C1848415041,0.00,0.00,1,0
681,1,CASH_OUT,20128.00,C1118430673,20128.00,0.00,C339924917,6268.00,12145.85,1,0
724,1,CASH_OUT,416001.33,C749981943,0.00,0.00,C667346055,102.00,9291619.62,1,0
969,1,TRANSFER,1277212.77,C1334405552,1277212.77,0.00,C431687661,0.00,0.00,1,0
970,1,CASH_OUT,1277212.77,C467632528,1277212.77,0.00,C716083600,0.00,2444985.19,1,0
1115,1,TRANSFER,35063.63,C1364127192,35063.63,0.00,C1136419747,0.00,0.00,1,0


In [53]:
dadosfinais = dadosfinais.sample(frac = 1)
dadosfinais = dadosfinais.reset_index()
dadosfinais.isFraud.value_counts(True)

0    0.752717
1    0.247283
Name: isFraud, dtype: float64

In [54]:
dadosfinais.columns

Index(['index', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [55]:
#utilizando o naive bayes retirado do site https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
#utilizando a mesma base de dados utilizada para treinar o naive bayes.
dadosteste = dadosfinais[:5000]
dadostreinamento = dadosfinais[5000:]
y = dadostreinamento.isFraud
x = dadostreinamento.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
z = dadosteste.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)
#help(model.predict)


[0 0 0 ..., 0 0 0]


In [56]:
#comparando o real com a classficação do naive bayes
dadosteste['predicted'] = pd.Series(predicted)
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())


0    3744
1    1256
Name: isFraud, dtype: int64
0    4471
1     529
Name: predicted, dtype: int64


C:\Users\Luca\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


rodar o naive bayes qualitativo (multinomial) e fazer falsos positivos, verdadeiros positivos... (proj2)